# 하나씩 분리해보는거

### 모델 로딩

In [1]:
import pybullet as p
import pybullet_data as pd
# 만약 env 안에 들어있는 애를 가져오고 싶으면 import env 해야함

import time

p.connect(p.GUI)
p.setAdditionalSearchPath(pd.getDataPath())

# localizing 시작
plane = p.loadURDF("plane.urdf")
p.setGravity(0, 0, -9.8)
p.setTimeStep(1. / 500)

#p.setDefaultContactERP(0)
#urdfFlags = p.URDF_USE_SELF_COLLISION+p.URDF_USE_SELF_COLLISION_EXCLUDE_ALL_PARENTS
urdfFlags = p.URDF_USE_SELF_COLLISION

# 만약 환경 바꾸고 싶으면 인식할꺼임
quadruped = p.loadURDF("laikago/laikago.urdf", [0, 0, .5], [0, 0.5, 0.5, 0],
                       flags=urdfFlags,
                       useFixedBase=False)


In [2]:
#### joint 정보 가지고 오는것

for j in range(p.getNumJoints(quadruped)):
    print(p.getJointInfo(quadruped, j))


(0, b'FR_hip_motor_2_chassis_joint', 0, 7, 6, 1, 0.0, 0.0, 0.0, -1.0, 100.0, 100.0, b'FR_hip_motor', (0.0, 0.0, -1.0), (-0.0817145, -0.03, 0.199095), (0.0, 0.0, 0.0, 1.0), -1)
(1, b'FR_upper_leg_2_hip_motor_joint', 0, 8, 7, 1, 0.0, 0.0, 0.0, -1.0, 100.0, 100.0, b'FR_upper_leg', (1.0, 0.0, 0.0), (-0.073565, 0.0, 0.0), (0.0, 0.0, 0.0, 1.0), 0)
(2, b'FR_lower_leg_2_upper_leg_joint', 0, 9, 8, 1, 0.0, 0.0, 0.0, -1.0, 100.0, 100.0, b'FR_lower_leg', (1.0, 0.0, 0.0), (0.02069, -0.16832999999999998, -0.10219999999999999), (0.0, 0.0, 0.0, 1.0), 1)
(3, b'FL_hip_motor_2_chassis_joint', 0, 10, 9, 1, 0.0, 0.0, 0.0, -1.0, 100.0, 100.0, b'FL_hip_motor', (0.0, 0.0, 1.0), (0.0817145, -0.03, 0.199095), (0.0, 0.0, 0.0, 1.0), -1)
(4, b'FL_upper_leg_2_hip_motor_joint', 0, 11, 10, 1, 0.0, 0.0, 0.0, -1.0, 100.0, 100.0, b'FL_upper_leg', (1.0, 0.0, 0.0), (0.075855, 0.0, 0.0), (0.0, 0.0, 0.0, 1.0), 3)
(5, b'FL_lower_leg_2_upper_leg_joint', 0, 12, 11, 1, 0.0, 0.0, 0.0, -1.0, 100.0, 100.0, b'FL_lower_leg', (1.0, 0

In [4]:
#enable collision between lower legs
#2,5,8 and 11 are the lower legs
lower_legs = [2, 5, 8, 11]
for l0 in lower_legs:
    for l1 in lower_legs:
        if (l1 > l0):
            enableCollision = 1
            print("collision for pair", l0, l1,
                p.getJointInfo(quadruped, l0)[12],
                p.getJointInfo(quadruped, l1)[12], "enabled=", enableCollision)
            p.setCollisionFilterPair(quadruped, quadruped, 2, 5, enableCollision)

jointIds = []
paramIds = []
jointOffsets = []
jointDirections = [-1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1]
jointAngles = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


collision for pair 2 5 b'FR_lower_leg' b'FL_lower_leg' enabled= 1
collision for pair 2 8 b'FR_lower_leg' b'RR_lower_leg' enabled= 1
collision for pair 2 11 b'FR_lower_leg' b'RL_lower_leg' enabled= 1
collision for pair 5 8 b'FL_lower_leg' b'RR_lower_leg' enabled= 1
collision for pair 5 11 b'FL_lower_leg' b'RL_lower_leg' enabled= 1
collision for pair 8 11 b'RR_lower_leg' b'RL_lower_leg' enabled= 1


In [5]:
for i in range(4):
    jointOffsets.append(0)
    jointOffsets.append(-0.7)
    jointOffsets.append(0.7)

maxForceId = p.addUserDebugParameter("maxForce", 0, 100, 20)

for j in range(p.getNumJoints(quadruped)):
    p.changeDynamics(quadruped, j, linearDamping=0, angularDamping=0)
    info = p.getJointInfo(quadruped, j)
    #print(info)
    jointName = info[1]
    jointType = info[2]
    if (jointType == p.JOINT_PRISMATIC or jointType == p.JOINT_REVOLUTE):
        jointIds.append(j)

p.getCameraImage(480, 320)
p.setRealTimeSimulation(0)

In [6]:
# 각각 joint가 실시간으로 움직이도록 값 주는부분

index = 0
for j in range(p.getNumJoints(quadruped)):
    p.changeDynamics(quadruped, j, linearDamping=0, angularDamping=0)
    info = p.getJointInfo(quadruped, j)
    js = p.getJointState(quadruped, j)
    #print(info)
    jointName = info[1]
    jointType = info[2]
    
    #print(jointDirections[index])
    if (jointType == p.JOINT_PRISMATIC or jointType == p.JOINT_REVOLUTE):
        paramIds.append(p.addUserDebugParameter(jointName.decode("utf-8"), -4, 4,
                                (js[0] - jointOffsets[index]) / jointDirections[index]))
    index = index+1
                       


In [ ]:
# simulation start

p.setRealTimeSimulation(1)

while (1):

    for i in range(len(paramIds)):
        c = paramIds[i]
        targetPos = p.readUserDebugParameter(c)
        maxForce = p.readUserDebugParameter(maxForceId)
        p.setJointMotorControl2(quadruped,
                            jointIds[i],
                            p.POSITION_CONTROL,
                            jointDirections[i] * targetPos + jointOffsets[i],
                            force=maxForce)

### 환경화 시키기

In [7]:
pybullet_client = p
p.connect(p.GUI)
p.setAdditionalSearchPath(pd.getDataPath())

plane = p.loadURDF("plane.urdf")
p.setGravity(0, 0, -9.8)
p.setTimeStep(1. / 500)
urdfFlags = p.URDF_USE_SELF_COLLISION

p.getCameraImage(480, 320)
p.setRealTimeSimulation(0)

timeStep = 1. / 500

In [1]:
import gym
import pybullet as p
import pybullet_data as pd

import time


from stable_baselines import PPO2
from stable_baselines.common.evaluation import evaluate_policy

/home/hayley/anaconda3/envs/benelot/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/hayley/anaconda3/envs/benelot/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/hayley/anaconda3/envs/benelot/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/hayley/anaconda3/envs/benelot/lib/pyt

In [1]:
import gym
import pybullet as p
import pybullet_data as pd

import time

from stable_baselines import PPO2
from stable_baselines.common.evaluation import evaluate_policy


from scene_stadium import SinglePlayerStadiumScene
from env_bases import MJCFBaseBulletEnv
import numpy as np
import pybullet
from robot_locomotors import Ant, Laikago


class WalkerBaseBulletEnv(MJCFBaseBulletEnv):

    def __init__(self, robot, render=False):
        # print("WalkerBase::__init__ start")
        self.camera_x = 0
        self.walk_target_x = 1e3  # kilometer away
        self.walk_target_y = 0
        self.stateId = -1
        MJCFBaseBulletEnv.__init__(self, robot, render)


    def create_single_player_scene(self, bullet_client):
        self.stadium_scene = SinglePlayerStadiumScene(bullet_client,
                                                  gravity=9.8,
                                                  timestep=0.0165 / 4,
                                                  frame_skip=4)
        return self.stadium_scene

    def reset(self):
        if (self.stateId >= 0):
            print("restoreState self.stateId:",self.stateId)
            self._p.restoreState(self.stateId)

        r = MJCFBaseBulletEnv.reset(self)
        self._p.configureDebugVisualizer(pybullet.COV_ENABLE_RENDERING, 0)

        # 여기가 ground plane 
        self.parts, self.jdict, self.ordered_joints, self.robot_body = self.robot.addToScene(
            self._p, self.stadium_scene.ground_plane_mjcf)
        
        
        # 여기가 joint 이름들
        self.ground_ids = set([(self.parts[f].bodies[self.parts[f].bodyIndex],
                            self.parts[f].bodyPartIndex) for f in self.foot_ground_object_names])
        
        
        
        
        self._p.configureDebugVisualizer(pybullet.COV_ENABLE_RENDERING, 1)
        #print("이거는 나오나")
        
        if (self.stateId < 0):
            self.stateId = self._p.saveState()
            print("saving state self.stateId:",self.stateId)

        return r

    def _isDone(self):
        return self._alive < 0

    def move_robot(self, init_x, init_y, init_z):
        
        "Used by multiplayer stadium to move sideways, to another running lane."
        self.cpp_robot.query_position()
        pose = self.cpp_robot.root_part.pose()
        
        pose.move_xyz(init_x, init_y, init_z)  
        # Works because robot loads around (0,0,0), and some robots have z != 0 that is left intact
        
        self.cpp_robot.set_pose(pose)

    electricity_cost = -2.0  # cost for using motors -- this parameter should be carefully tuned against reward for making progress, other values less improtant
    stall_torque_cost = -0.1  # cost for running electric current through a motor even at zero rotational speed, small
    foot_collision_cost = -1.0  # touches another leg, or other objects, that cost makes robot avoid smashing feet into itself
    foot_ground_object_names = set(["floor"])  # to distinguish ground and other objects
    joints_at_limit_cost = -0.1  # discourage stuck joints

    def step(self, a):
        
        if not self.scene.multiplayer:  # if multiplayer, action first applied to all robots, then global step() called, then _step() for all robots with the same actions
            self.robot.apply_action(a)
            self.scene.global_step()

        state = self.robot.calc_state()  # also calculates self.joints_at_limit

        self._alive = float(self.robot.alive_bonus(
                state[0] + self.robot.initial_z,
                self.robot.body_rpy[1]))  
                # state[0] is body height above ground, body_rpy[1] is pitch
            
        done = self._isDone()
        
        if not np.isfinite(state).all():
            print("~INF~", state)
            done = True

        potential_old = self.potential
        self.potential = self.robot.calc_potential()
        progress = float(self.potential - potential_old)

        feet_collision_cost = 0.0
        
        for i, f in enumerate(self.robot.feet):  
            # TODO: Maybe calculating feet contacts could be done within the robot code
            
            contact_ids = set((x[2], x[4]) for x in f.contact_list())
            #print("CONTACT OF '%d' WITH %d" % (contact_ids, ",".join(contact_names)) )
            #print(contact_ids)
            
            if (self.ground_ids & contact_ids):
                #see Issue 63: https://github.com/openai/roboschool/issues/63
                #feet_collision_cost += self.foot_collision_cost
                self.robot.feet_contact[i] = 1.0
                
            else:
                self.robot.feet_contact[i] = 0.0

        electricity_cost = self.electricity_cost * float(np.abs(a * self.robot.joint_speeds).mean())
        electricity_cost += self.stall_torque_cost * float(np.square(a).mean())

        joints_at_limit_cost = float(self.joints_at_limit_cost * self.robot.joints_at_limit)
        
        debugmode = 0
        if (debugmode):
            print("alive=")
            print(self._alive)
            print("progress")
            print(progress)
            print("electricity_cost")
            print(electricity_cost)
            print("joints_at_limit_cost")
            print(joints_at_limit_cost)
            print("feet_collision_cost")
            print(feet_collision_cost)

        self.rewards = [
            self._alive, progress, electricity_cost, joints_at_limit_cost, feet_collision_cost
        ]
        
        if (debugmode):
            print("rewards=")
            print(self.rewards)
            print("sum rewards")
            print(sum(self.rewards))
            
        self.HUD(state, a, done)
        self.reward += sum(self.rewards)

        return state, sum(self.rewards), bool(done), {}

    def camera_adjust(self):
        x, y, z = self.body_xyz
        self.camera_x = 0.98 * self.camera_x + (1 - 0.98) * x
        self.camera.move_and_look_at(self.camera_x, y - 2.0, 1.4, x, y, 1.0)




class AntBulletEnv(WalkerBaseBulletEnv):

    def __init__(self, render=False):
        self.robot = Ant()
        WalkerBaseBulletEnv.__init__(self, self.robot, render)

class LaikagoBulletEnv(WalkerBaseBulletEnv):

    def __init__(self, render=False):
        self.robot = Laikago()
        WalkerBaseBulletEnv.__init__(self, self.robot, render)



/home/hayley/anaconda3/envs/benelot/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/hayley/anaconda3/envs/benelot/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/hayley/anaconda3/envs/benelot/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/hayley/anaconda3/envs/benelot/lib/pyt

In [2]:
env = AntBulletEnv() 

/home/hayley/.mujoco/gym/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [3]:
env.render()

array([], dtype=float64)

In [4]:
observation = env.reset()

/home/hayley/.mujoco/stable-baseline/data/ant.xml
((0.0, 0.0, 0.0), (0.0, 0.0, 0.0, 1.0))
((0.0, 0.0, 0.75), (0.0, 0.0, 0.0, 1.0))
saving state self.stateId: 0


#### 여기서부터 training

In [5]:
model = PPO2('MlpPolicy', env, gamma=0.99, n_steps=256, ent_coef=0.0, learning_rate=2.5e-4, 
              lam=0.95, nminibatches=32, noptepochs=10, cliprange=0.2,  verbose=1)

Wrapping the env in a DummyVecEnv.




Instructions for updating:
Use keras.layers.flatten instead.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




In [6]:
print(env)

<AntBulletEnv instance>


In [ ]:
model.learn(total_timesteps=10000)

restoreState self.stateId: 0
[0. 0. 0. 0.]
-------------------------------------
| approxkl           | 0.008263671  |
| clipfrac           | 0.103515625  |
| explained_variance | 0.0332       |
| fps                | 164          |
| n_updates          | 1            |
| policy_entropy     | 11.345223    |
| policy_loss        | -0.015184489 |
| serial_timesteps   | 256          |
| time_elapsed       | 2.62e-06     |
| total_timesteps    | 256          |
| value_loss         | 20.09955     |
-------------------------------------
restoreState self.stateId: 0
[0. 0. 0. 0.]
------------------------------------
| approxkl           | 0.012049732 |
| clipfrac           | 0.17539063  |
| explained_variance | -0.0177     |
| fps                | 223         |
| n_updates          | 2           |
| policy_entropy     | 11.35161    |
| policy_loss        | -0.02859874 |
| serial_timesteps   | 512         |
| time_elapsed       | 1.56        |
| total_timesteps    | 512         |
| value_loss 

In [8]:
# Save the agent
model.save("ant_trained")
del model  # delete trained model to demonstrate loading

In [9]:
# Load the trained agent
model = PPO2.load("/home/hayley/.mujoco/stable-baseline/ant_trained")

# Enjoy trained agent
obs = env.reset()
for i in range(10000):
    action, _states = model.predict(obs)
    obs, rewards, dones, info = env.step(action)
    env.render()

Loading a model without an environment, this model cannot be trained until it has a valid environment.
restoreState self.stateId: 0


#### Laikago

In [1]:
import gym
import pybullet as p
import pybullet_data as pd

import time

from stable_baselines import PPO2
from stable_baselines.common.evaluation import evaluate_policy


from scene_stadium import SinglePlayerStadiumScene
from env_bases import MJCFBaseBulletEnv
import numpy as np
import pybullet
from robot_locomotors import Ant, Laikago


class WalkerBaseBulletEnv(MJCFBaseBulletEnv):

    def __init__(self, robot, render=False):
        # print("WalkerBase::__init__ start")
        self.camera_x = 0
        self.walk_target_x = 1e3  # kilometer away
        self.walk_target_y = 0
        self.stateId = -1
        MJCFBaseBulletEnv.__init__(self, robot, render)

    
    #0.0165 / 4
    #1./500
    def create_single_player_scene(self, bullet_client):
        self.stadium_scene = SinglePlayerStadiumScene(bullet_client,
                                                  gravity=9.8,
                                                  timestep=1./500, 
                                                  frame_skip=4)
        return self.stadium_scene

    def reset(self):
        if (self.stateId >= 0):
            #print("restoreState self.stateId:",self.stateId)
            self._p.restoreState(self.stateId)

        
        r = MJCFBaseBulletEnv.reset(self)
        self._p.configureDebugVisualizer(pybullet.COV_ENABLE_RENDERING, 0)

        # 여기가 ground plane 
        self.parts, self.jdict, self.ordered_joints, self.robot_body = self.robot.addToScene(
            self._p, self.stadium_scene.ground_plane_mjcf)
        
        
        # 여기가 joint 이름들
        self.ground_ids = set([(self.parts[f].bodies[self.parts[f].bodyIndex],
                            self.parts[f].bodyPartIndex) for f in self.foot_ground_object_names])
        #print(self.ground_ids)
        
        self._p.configureDebugVisualizer(pybullet.COV_ENABLE_RENDERING, 1)
        
        
        if (self.stateId < 0):
            self.stateId = self._p.saveState()
            print("saving state self.stateId:",self.stateId)
            

        return r

    def _isDone(self):
        return self._alive < 0

    def move_robot(self, init_x, init_y, init_z):
        
        "Used by multiplayer stadium to move sideways, to another running lane."
        self.cpp_robot.query_position()
        pose = self.cpp_robot.root_part.pose()
        #print(pose)
        
        pose.move_xyz(init_x, init_y, init_z)  
        # Works because robot loads around (0,0,0), and some robots have z != 0 that is left intact
        
        self.cpp_robot.set_pose(pose)

    electricity_cost = -2.0  # cost for using motors -- this parameter should be carefully tuned against reward for making progress, other values less improtant
    stall_torque_cost = -0.1  # cost for running electric current through a motor even at zero rotational speed, small
    foot_collision_cost = -1.0  # touches another leg, or other objects, that cost makes robot avoid smashing feet into itself
    foot_ground_object_names = set(["floor"])  # to distinguish ground and other objects
    joints_at_limit_cost = -0.1  # discourage stuck joints

    def step(self, a):
        
        if not self.scene.multiplayer:  # if multiplayer, action first applied to all robots, then global step() called, then _step() for all robots with the same actions
            #print("not multiplayer")
            self.robot.apply_action(a)
            self.scene.global_step()

        state = self.robot.calc_state()  # also calculates self.joints_at_limit

        self._alive = float(self.robot.alive_bonus(
                state[0] + self.robot.initial_z,
                self.robot.body_rpy[1]))  
                # state[0] is body height above ground, body_rpy[1] is pitch
            
        done = self._isDone()
        
        if not np.isfinite(state).all():
            print("~INF~", state)
            done = True

        potential_old = self.potential
        self.potential = self.robot.calc_potential()
        progress = float(self.potential - potential_old)

        feet_collision_cost = 0.0
        
        for i, f in enumerate(self.robot.feet):  
            # TODO: Maybe calculating feet contacts could be done within the robot code
            
            contact_ids = set((x[2], x[4]) for x in f.contact_list())
            #print("CONTACT OF '%d' WITH %d" % (contact_ids, ",".join(contact_names)) )
            #print(contact_ids)
            
            if (self.ground_ids & contact_ids):
                #see Issue 63: https://github.com/openai/roboschool/issues/63
                #feet_collision_cost += self.foot_collision_cost
                self.robot.feet_contact[i] = 1.0
                
            else:
                self.robot.feet_contact[i] = 0.0

        electricity_cost = self.electricity_cost * float(np.abs(a * self.robot.joint_speeds).mean())
        electricity_cost += self.stall_torque_cost * float(np.square(a).mean())

        joints_at_limit_cost = float(self.joints_at_limit_cost * self.robot.joints_at_limit)
        
        debugmode = 0
        if (debugmode):
            print("alive=")
            print(self._alive)
            print("progress")
            print(progress)
            print("electricity_cost")
            print(electricity_cost)
            print("joints_at_limit_cost")
            print(joints_at_limit_cost)
            print("feet_collision_cost")
            print(feet_collision_cost)

        self.rewards = [
            self._alive, progress, electricity_cost, joints_at_limit_cost, feet_collision_cost
        ]
        
        if (debugmode):
            print("rewards=")
            print(self.rewards)
            print("sum rewards")
            print(sum(self.rewards))
            
        self.HUD(state, a, done)
        self.reward += sum(self.rewards)

        return state, sum(self.rewards), bool(done), {}

    def camera_adjust(self):
        x, y, z = self.body_xyz
        self.camera_x = 0.98 * self.camera_x + (1 - 0.98) * x
        self.camera.move_and_look_at(self.camera_x, y - 2.0, 1.4, x, y, 1.0)




class AntBulletEnv(WalkerBaseBulletEnv):

    def __init__(self, render=False):
        self.robot = Ant()
        WalkerBaseBulletEnv.__init__(self, self.robot, render)

class LaikagoBulletEnv(WalkerBaseBulletEnv):

    def __init__(self, render=False):
        self.robot = Laikago()
        WalkerBaseBulletEnv.__init__(self, self.robot, render)



/home/hayley/anaconda3/envs/benelot/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/hayley/anaconda3/envs/benelot/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/hayley/anaconda3/envs/benelot/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/hayley/anaconda3/envs/benelot/lib/pyt

In [2]:
env = LaikagoBulletEnv() 

# register 하는거는 옵션이라서 이거 없어도 되야함

/home/hayley/.mujoco/gym/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [3]:
env.render()

print(env.observation_space)
print(env.action_space)
print(env.action_space.sample())

Box(36,)
Box(12,)
[-0.25802284 -0.42228353 -0.36002916  0.6077592   0.40304548 -0.67905766
  0.89339375  0.7148967   0.9536847   0.8949034  -0.04416413 -0.39381394]


In [4]:
observation = env.reset()

/home/hayley/.mujoco/stable-baseline/laikago/laikago.urdf
((0.0, 0.0, 0.0), (0.0, 0.0, 0.0, 1.0))
13
saving state self.stateId: 0


In [5]:
model = PPO2('MlpPolicy', env, gamma=0.99, n_steps=256, ent_coef=0.0, learning_rate=2.5e-4, 
              lam=0.95, nminibatches=32, noptepochs=10, cliprange=0.2,  verbose=1, tensorboard_log="./laikago_trained_log/")

Wrapping the env in a DummyVecEnv.




Instructions for updating:
Use keras.layers.flatten instead.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




In [6]:
print(env)

<LaikagoBulletEnv instance>


In [ ]:
# 앞으로 좀 걸어나가야 하는데 제자리에서만 움직임 (뭔가 컨트롤러가 땅을 집고 못 걷는 느낌?)

model.learn(total_timesteps=10000)


-------------------------------------
| approxkl           | 0.018927079  |
| clipfrac           | 0.30664062   |
| explained_variance | 0.000368     |
| fps                | 26           |
| n_updates          | 1            |
| policy_entropy     | 16.91876     |
| policy_loss        | -0.056863666 |
| serial_timesteps   | 256          |
| time_elapsed       | 2.62e-06     |
| total_timesteps    | 256          |
| value_loss         | 21.564743    |
-------------------------------------


In [ ]:
# Save the agent
model.save("laikago_trained")
del model  # delete trained model to demonstrate loading

In [ ]:
# Load the trained agent
model = PPO2.load("/home/hayley/.mujoco/stable-baseline/laikago_trained")

# Enjoy trained agent
obs = env.reset()
for i in range(10000):
    action, _states = model.predict(obs)
    obs, rewards, dones, info = env.step(action)
    env.render()

Loading a model without an environment, this model cannot be trained until it has a valid environment.
